In [16]:
import pandas as pd
df=pd.read_csv('cleaned.csv')

In [18]:
df.drop ('Unnamed: 0', axis=1, inplace=True)
df

,sources,titles,contents,date,authors,urls,texts,cleaned_text
0,Associated Press,Guest lineups for the Sunday news shows,WASHINGTON (AP) Guest lineups for the Sunday n...,2020-03-27T22:56:58Z,NaN,https://apnews.com/beafbe1d484050ac97438c7bd6a...,Guest lineups for the Sunday news shows\n\nWAS...,guest lineups for the sunday news shows washi...
1,Associated Press,Country singer Joe Diffie tests positive for c...,"NASHVILLE, Tenn. (AP) A publicist for Joe Diff...",2020-03-27T19:33:49Z,By KRISTIN M. HALL AP Entertainment Writer,https://apnews.com/3e2edb7ce690346df645d79dae4...,"FILE - This Aug. 22, 2018 file photo shows Joe...","file - this aug. 22, 2018 file shows joe dif..."
2,Associated Press,Milwaukee's black community hit hard by corona...,"MADISON, Wis. (AP) Deaths and confirmed cases ...",2020-03-27T17:22:54Z,By SCOTT BAUER Associated Press,https://apnews.com/b52e4e9a63d64e3a25109f09010...,The onramp to the Hoan Bridge is void of traff...,the onramp to the hoan bridge is void of traff...
3,Associated Press,"Column: As we battle virus, don't forget 1919 ...","After a week of travel, which included a coupl...",2020-03-27T21:47:11Z,PAUL NEWBERRY AP Sports Columnist,https://apnews.com/6f1fef1bcf19f9b0eb9cd8bce39...,"FILE - In this Feb. 4, 2005, file photo, the i...","file - in this feb. 4, 2005, file , the inscr..."
4,Breitbart News,"Joe Rogan Worried by 'Old,' 'Stumbling' Biden:...",Podcast host Joe Rogan blasted the Democrat Pa...,2020-03-27T15:29:30Z,"Hannah Bleau, Hannah Bleau",https://www.breitbart.com/politics/2020/03/27/...,Podcast host Joe Rogan blasted the Democrat Pa...,podcast host joe rogan blasted the democrat pa...
...,...,...,...,...,...,...,...,...
216,Time,President Trump and China’s Xi Pledge to Coope...,U.S. President Donald Trump and China’s Xi Jin...,2020-03-27T10:10:35Z,Karen Leigh / Bloomberg,https://time.com/5811238/trump-xi-cooperate-co...,U.S. President Donald Trump and China’s Xi Jin...,u.s. president donald trump and china’s xi jin...
217,Time,Report of Urns Stacked at Wuhan Funeral Homes ...,(Bloomberg) — The long lines and stacks of ash...,2020-03-27T08:05:28Z,Bloomberg,https://time.com/5811222/wuhan-coronavirus-dea...,(Bloomberg) — The long lines and stacks of ash...,(bloomberg) — the long lines and stacks of ash...
218,Time,Trump’s Iran Gamble: Will “Maximum Pressure” A...,"On a recent evening in Tehran, apartment dwell...",2020-03-27T21:19:21Z,Kimberly Dozier and John Walcott,https://time.com/5811772/trumps-iran-gamble/,"On a recent evening in Tehran, apartment dwell...","on a recent evening in tehran, apartment dwell..."
219,New York Magazine,How Will New York’s Rent Freeze Work?,Were committed to keeping our readers informed...,2020-03-27T22:34:57Z,James D. Walsh,http://nymag.com/intelligencer/2020/03/how-wil...,Photo: Craig Warga/Bloomberg via Getty Images\...,: craig warga/bloomberg via we’re commit...


Getting TextBlob score for every article:

In [19]:
from textblob import TextBlob
textblob_score=[]
for i in df['cleaned_text']:
    score=TextBlob(i)
    polarity=score.sentiment.polarity
    textblob_score.append(polarity)

Google sentiment analyzer Score and entities score:

1-Google API authentication:
following the steps in this link (https://cloud.google.com/natural-language/docs/sentiment-tutorial)

In [20]:
import os
# setiing GOOGLE_APPLICATION_CREDENTIALS environment variable value to the location where the application key was stored
credential_path = "C:/Users/suzan/Documents/Downloads/google_key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path


Defining the function which can call Google API and retrieve its sentiment score for any text:

In [66]:
import argparse
from google.cloud import language_v1
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

def google_sentiment(cleaned_article_text):
    """Run a sentiment analysis request on text """
    
    #creaating a client object
    client = language.LanguageServiceClient()

    content=cleaned_article_text

    #content type is set to PLAIN_TEXT
    document = types.Document(
        content=content,
        type=enums.Document.Type.PLAIN_TEXT)
    
    #calling analyze_sentiment method on this object
    annotations = client.analyze_sentiment(document=document)

    #annotations stores the score under document_sentiment
    score_result=annotations.document_sentiment.score
    return score_result


In [47]:
# an example:
google_sentiment=google_sentiment(' queen elizabeth has been in self-isolation at windsor castle in order to avoid contracting coronavirus')
google_sentiment

0.20000000298023224

In [67]:
google_score=[]
for i in df['cleaned_text']:

    score=google_sentiment(i)
    google_score.append(score)

Google entity sentiment analyzer function:

In [53]:
def google_analyze_entity_sentiment(text_content_arg):
    """
    Analyzing Entity Sentiment in a String

    Args:
      text_content_arg The text content to analyze in this case it is the articles text
    """
    
    # to have consistent names for the candidates entities (the entity will be their full name):
    text_content=text_content_arg.lower()
    text_content=text_content.replace('elizabeth', 'elizabeth warren').replace('elizabeth warren warren', 'elizabeth warren')
    text_content=text_content.replace('pete', 'pete buttigieg').replace('pete buttigieg buttigieg', 'pete buttigieg')
    text_content=text_content.replace('michael', 'michael bloomberg').replace('michael bloomberg bloomberg', 'michael bloomberg')
    text_content=text_content.replace('bernie', 'bernie sanders').replace('bernie sanders sanders', 'bernie sanders')
    text_content=text_content.replace('joe', 'joe biden').replace('joe biden biden', 'joe biden')

    #creating the client object
    client = language_v1.LanguageServiceClient()

    # text_content = 'Grapes are good. Bananas are bad.'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}
    
    # listing the entities we want to extract the scores for:
    candidates=['senator bernie sanders','senator sanders','senator bernie','bernie sanders','bernie', 'sanders','joe biden','joe', 'biden','vice president','vp joe biden',
                'vp biden','vp joe','vice president joe biden','vice president biden','vice president joe'
                'senator elizabeth warren','elizabeth warren','elizabeth','senator warren', 'warren','senator elizabeth'
                'mayor pete','mayor pete buttigieg','mayor buttigieg','pete buttigieg', 'pete', 'buttigieg',
                'michael bloomberg','michael','bloomberg','mike','mike bloomberg']
    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8
    entities=[]
    score=[]
    
    #The returned response for the client
    response = client.analyze_entity_sentiment(document, encoding_type=encoding_type)
    # Loop through entitites returned from the API
    for entity in response.entities:

        res = [ele for ele in candidates if(ele in entity.name)] 
        if (res):
            entities.append(entity.name)

            sentiment = entity.sentiment

            score.append(sentiment.score)



    return entities,score


In [29]:
# examples:

s1=google_analyze_entity_sentiment('Joe is a good candidate')
# score for Joe :
print(s1)
s2=google_analyze_entity_sentiment('Joe is a good candidate but Bernie has more supporters among the young people')
# scores for Joe and Bernie respectively :
print(s2)
s3=google_analyze_entity_sentiment('Donald Trump has high chances of being reelected')
# Nothing will be printed or returned for Trump, because in the function we filtered on 5 entities (last 5 candidates in the US democratic party elections)
print(s3)

(['joe biden'], [0.8999999761581421])
(['joe biden', 'bernie sanders'], [0.800000011920929, 0.10000000149011612])
([], [])


listing the possible mention syntax for our target entities:

In [54]:
bernielist=['senator bernie sanders','senator sanders','senator bernie','bernie sanders','bernie', 'sanders']
joelist=['joe biden','joe', 'biden','vice president','vp joe biden',
                'vp biden','vp joe','vice president joe biden','vice president biden','vice president joe']

warrenlist=['senator elizabeth warren','elizabeth warren','elizabeth','senator warren', 'warren','senator elizabeth']

petelist=['mayor pete','mayor pete buttigieg','mayor buttigieg','pete buttigieg', 'pete', 'buttigieg']
mikelist=['michael bloomberg','michael','bloomberg','mike','mike bloomberg']

looping over the texts to get each entity score in all articles:

In [56]:
google_joesentiment=[]
google_berniesentiment=[]
google_warrensentiment=[]
google_petesentiment=[]
google_mikesentiment=[]
for j in df['cleaned_text']:
    obj1=sample_analyze_entity_sentiment(j)
    if (len(obj1[0])==0):
  
        google_joesentiment.append(0)
      
        google_berniesentiment.append(0)
  
        google_warrensentiment.append(0)
    
        google_petesentiment.append(0)

        google_mikesentiment.append(0)
      
    joescore=0
    berniescore=0
    warrenscore=0
    petescore=0
    mikescore=0
        
    for entity in range(0,len(obj1[0])):
        if (obj1[0][entity] in joelist):
          
            joescore=+obj1[1][entity]

        elif (obj1[0][entity] in bernielist):
          
            berniescore=+obj1[1][entity]      
    
        elif (obj1[0][entity] in warrenlist):
          
            warrenscore=+obj1[1][entity]
           
        elif (obj1[0][entity] in petelist):
        
            petescore=+obj1[1][entity]
     
        elif (obj1[0][entity] in mikelist):
           
            mikescore=+obj1[1][entity]
    
            
    if (len(obj1[0])>0):
        google_joesentiment.append(joescore)
        
        google_berniesentiment.append(berniescore)
       
        google_warrensentiment.append(warrenscore)
      
        google_petesentiment.append(petescore)
       
        google_mikesentiment.append(mikescore)
    

In [71]:
# adding the columns to the dataframe
df['textblob_score']=textblob_score
df['google_score']=google_score
df['google_joe']=google_joesentiment
df['google_bernie']=google_berniesentiment
df['google_warren']=google_warrensentiment
df['google_pete']=google_petesentiment
df['google_mike']=google_mikesentiment
df.head()

,sources,titles,contents,date,authors,urls,texts,cleaned_text,textblob_score,google_score,google_joe,google_bernie,google_warren,google_pete,google_mike
0,Associated Press,Guest lineups for the Sunday news shows,WASHINGTON (AP) Guest lineups for the Sunday n...,2020-03-27T22:56:58Z,NaN,https://apnews.com/beafbe1d484050ac97438c7bd6a...,Guest lineups for the Sunday news shows\n\nWAS...,guest lineups for the sunday news shows washi...,0.026263,0.1,0.0,0.0,0.0,0.0,0.0
1,Associated Press,Country singer Joe Diffie tests positive for c...,"NASHVILLE, Tenn. (AP) A publicist for Joe Diff...",2020-03-27T19:33:49Z,By KRISTIN M. HALL AP Entertainment Writer,https://apnews.com/3e2edb7ce690346df645d79dae4...,"FILE - This Aug. 22, 2018 file photo shows Joe...","file - this aug. 22, 2018 file shows joe dif...",0.146571,0.1,0.0,0.0,0.0,0.0,0.0
2,Associated Press,Milwaukee's black community hit hard by corona...,"MADISON, Wis. (AP) Deaths and confirmed cases ...",2020-03-27T17:22:54Z,By SCOTT BAUER Associated Press,https://apnews.com/b52e4e9a63d64e3a25109f09010...,The onramp to the Hoan Bridge is void of traff...,the onramp to the hoan bridge is void of traff...,0.026944,-0.1,0.0,0.0,0.0,0.0,0.0
3,Associated Press,"Column: As we battle virus, don't forget 1919 ...","After a week of travel, which included a coupl...",2020-03-27T21:47:11Z,PAUL NEWBERRY AP Sports Columnist,https://apnews.com/6f1fef1bcf19f9b0eb9cd8bce39...,"FILE - In this Feb. 4, 2005, file photo, the i...","file - in this feb. 4, 2005, file , the inscr...",-0.018783,0.0,0.0,0.0,0.0,0.0,0.0
4,Breitbart News,"Joe Rogan Worried by 'Old,' 'Stumbling' Biden:...",Podcast host Joe Rogan blasted the Democrat Pa...,2020-03-27T15:29:30Z,"Hannah Bleau, Hannah Bleau",https://www.breitbart.com/politics/2020/03/27/...,Podcast host Joe Rogan blasted the Democrat Pa...,podcast host joe rogan blasted the democrat pa...,0.051391,-0.2,-0.5,-0.3,0.0,0.0,0.0


In [72]:
#saving everything into a csv file
df.to_csv('all_scores.csv')